In [ ]:
import json

from specklepy.api.client import SpeckleClient
from specklepy.core.api import operations
from specklepy.transports.server import ServerTransport

# Leapfrog Weka Hills Demo Model from Speckle to IFC 5

In [ ]:
weka_hills_speckle_project_id = "7a489ac0d4"
ifc5_geo_dict = {"name": "Alluvium", "ifc5_ref": "Ne4ef1ea1101a484183bb8a716924702e"}
# ifc5_geo_dict = {"name": "Siltstone", "ifc5_ref": "N4586ef982fd34faab3de532a7a43544d"}
# ifc5_geo_dict = {"name": "Basement", "ifc5_ref": "N450a532f5e414d609856a8a50b6a94ad"}

In [ ]:
speckle_client = SpeckleClient(host="app.speckle.systems")
transport = ServerTransport(
    stream_id=weka_hills_speckle_project_id, client=speckle_client
)

In [ ]:
weka_hills_speckle_models = speckle_client.model.get_models(
    project_id=weka_hills_speckle_project_id
)

geology_models_w_versions = {}
for model in weka_hills_speckle_models.items:
    if "geo/geology-model" in model.name:
        geological_unit = str(model.name).replace("geo/geology-model/", "")
        speckle_model = speckle_client.model.get_with_versions(
            model_id=model.id,
            project_id=weka_hills_speckle_project_id,
        )
        geology_models_w_versions[geological_unit] = speckle_model
        print(f"{geological_unit}: {model.id}")

In [ ]:
model_data = operations.receive(
    geology_models_w_versions[ifc5_geo_dict["name"].lower()]
    .versions.items[0]
    .referencedObject,
    transport,
)

In [ ]:
speckle_faces = model_data.elements[0].elements[0].elements[0].elements[0].faces
face_vertex_counts = []
face_vertex_indices = []
i = 0
while i < len(speckle_faces):
    vertex_count = speckle_faces[i]
    face_vertex_counts.append(vertex_count)
    face_vertex_indices.extend(speckle_faces[i + 1 : i + 1 + vertex_count])
    i += 1 + vertex_count

speckle_vertices = model_data.elements[0].elements[0].elements[0].elements[0].vertices
points = [speckle_vertices[i : i + 3] for i in range(0, len(speckle_vertices), 3)]

In [ ]:
ifc5_usdgeom = [
    {
        "def": "over",
        "name": f"{ifc5_geo_dict['ref']}_Body",
        "attributes": {
            "UsdGeom:Mesh": {
                # "faceVertexCounts": face_vertex_counts,
                "faceVertexIndices": face_vertex_indices,
                "points": points,
            }
        },
    }
]

with open(
    f"WekaHills_GeologyModel_{ifc5_geo_dict['name']}_UsdGeom.ifcx", "w"
) as json_file:
    json.dump(ifc5_usdgeom, json_file)

For testing purposes, one can create an `.obj` file from the Speckle geometry as follows:

In [ ]:
with open(f"{geological_unit}Mesh.obj", "w") as obj_file:
    for point in points:
        obj_file.write(f"v {point[0]} {point[1]} {point[2]}\n")
    for i in range(0, len(face_vertex_indices), 3):
        # OBJ uses 1-based indexing, so add 1 to the indices
        obj_file.write(
            f"f {face_vertex_indices[i] + 1} {face_vertex_indices[i + 1] + 1} {face_vertex_indices[i + 2] + 1}\n"
        )